In [1]:
from tope import Tope
from tope.net import *
from tope.orth import *
from tests import normalize_polygon
from typing import *

import json, os, numpy as np
rng = np.random.default_rng()
with open("polys.json") as fd: polys = json.load(fd)

In [4]:
v_24cell = [
    [1, 0, 0, 0], [-1, 0, 0, 0],
    [0, 1, 0, 0], [0, -1, 0, 0],
    [0, 0, 1, 0], [0, 0, -1, 0], 
    [0, 0, 0, 1], [0, 0, 0, -1],
    [1, -1, 0, 0], [-1, 1, 0, 0], 
    [1, 0, -1, 0], [-1, 0, 1, 0], 
    [1, 0, 0, -1], [-1, 0, 0, 1], 
    [0, 1, -1, 0], [0, -1, 1, 0], 
    [0, 1, 0, -1], [0, -1, 0, 1],
    [1, 0, -1, -1], [-1, 0, 1, 1], 
    [0, 1, -1, -1], [0, -1, 1, 1],
    [1, 1, -1, -1], [-1, -1, 1, 1]
]

In [5]:
v_36cell = [
    (1, 0, 0, 0),
    (0, 0, 1, 0),
    (0, -1, 0, 0),
    (0, 0, 0, -1),
    (1, 1, 0, 0),
    (-1, -1, 0, 0),
    (0, 1, 0, 0),
    (-1, 0, 0, 0),
    (0, 0, 1, 1),
    (0, 0, 0, 1),
    (0, 0, -1, -1),
    (0, 0, -1, 0)
]

In [12]:
edges = [[0, 2], [8, 2], [8, 5], [9, 2], [9, 5], [2, 11], [11, 5], [10, 2], [10, 5], [5, 7], [1, 2], [2, 3], [2, 5], [1, 5], [3, 5], [10, 7], [11, 7], [3, 7], [9, 7], [1, 7], [8, 7], [6, 7], [8, 6], [1, 6], [9, 6], [3, 6], [11, 6], [10, 6], [4, 6], [10, 11], [10, 3], [0, 10], [10, 4], [9, 11], [0, 11], [11, 4], [1, 3], [0, 3], [3, 4], [8, 9], [0, 9], [9, 4], [8, 1], [0, 1], [1, 4], [0, 8], [8, 4], [0, 4]]
edges = [[i+1 for i in e] for e in edges]
print(edges)

[[1, 3], [9, 3], [9, 6], [10, 3], [10, 6], [3, 12], [12, 6], [11, 3], [11, 6], [6, 8], [2, 3], [3, 4], [3, 6], [2, 6], [4, 6], [11, 8], [12, 8], [4, 8], [10, 8], [2, 8], [9, 8], [7, 8], [9, 7], [2, 7], [10, 7], [4, 7], [12, 7], [11, 7], [5, 7], [11, 12], [11, 4], [1, 11], [11, 5], [10, 12], [1, 12], [12, 5], [2, 4], [1, 4], [4, 5], [9, 10], [1, 10], [10, 5], [9, 2], [1, 2], [2, 5], [1, 9], [9, 5], [1, 5]]


In [ ]:
EdgeList = list[np.ndarray] # list of 2xdim arrays

def get_cuts(N: Net) -> EdgeList:
    return np.concatenate([N.tope.meta[2][i]["cuts"] for i in range(len(N.facets))])
        
        
def get_edges(N: Net) -> EdgeList: # apply to unfolded Net
    edges = []
    for i, vertices in N.facets.items():
        facet_template = N.tope.get_face(i) # has correct indices
        edges.extend((vertices[list(e)] for e in facet_template.faces[1]))
    return edges

import itertools

Net2d = None # new format of Net

def iter_edges(N: Net2d) -> Iterable[np.ndarray[2,2]]: # apply to unfolded 2d Net
    return N.iter_faces_as_vertices(dim=1)

FacetLabels = list[tuple[str, np.ndarray]] # label, position

def get_facet_labels(N: Net) -> FacetLabels:
    labels = []
    for i, vertices in N.facets.items():
        labels.append((N.tope.meta[N.tope.dim-1][i]["index"], vertices.mean(axis=0)))
    return labels

def iter_facet_labels(N: Net2d, key: str) -> Iterable[str]:
    return zip(N.iter_meta(dim=2, key="index"), map(N.cells.values(), lambda x: x.vertices.mean(axis=0)))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection # container for line segments
from matplotlib.text import Text, Annotation
import matplotlib.cm as cm # colour mappings
import matplotlib as mpl

In [ ]:
def get_net_for_facet(P: Tope, i: int, keys=[]) -> Net:
    F = P.get_facet(i, meta_keys=keys)
    G = get_facet_graph(F)
    T = G.get_spanning_tree()
    N = Net(F, T).unfold_with_meta(meta_keys=keys).in_own_span(meta_keys=keys)
    return N

In [ ]:
MARGIN_FACTOR = 0.05

from tope.plot import bounding_bbox, plot_artists_in_view

def plot_nets(
    P: Tope, 
    name: str = "P", 
    margin: float = MARGIN_FACTOR, 
    dpi: int = 300, 
    force=False, 
    fmt="svg",
    label_config: dict = {"fontsize": 2, "ha": "center", "va": "center"},
    title_config: dict = {"fontsize": 8, "pad": -14},
    hatch = False,
    label = True
):    
    nfacets = len(P.faces[P.dim-1])
    nets = [get_net_for_facet(P, i, keys=["cuts"]) for i in range(nfacets)] if hatch \
        else [get_net_for_facet(P, i) for i in range(nfacets)]
    
    ## compute common bounding box
    
    bbox = bounding_bbox(*(cell for N in nets for cell in N.facets.values()))
    
    #xlim, ylim = compute_bounding_box(*(cell for N in nets for cell in N.facets.values()), margin_factor=margin_factor)
    
    ## generate images
    figs = []
    for i, N in enumerate(nets):
        artists = [LineCollection(get_edges(N), colors="grey", linewidths=0.2)]
        
        if hatch: artists.append(
            LineCollection(get_cuts(N), colors="lightgrey", linewidths=0.05)
        )
            
        if label: artists.extend(
            Text(*pos, text=str(l), **label_config) for l, pos in get_facet_labels(N)
        )
        
        # draw the actual picture
        fig, ax = plot_artists_in_view(*artists, bbox = bbox, margin = margin)
        
        facet_name = f"{name}-facet-{i}"
        # title with "default" positioning
        ax.set_title(facet_name, **title_config)
                
        # store and next()
        figs.append((facet_name, fig))
    
    return figs

In [ ]:
def get_hyperplanes(*planes, spacing=0.02, r=5):
    return [H for L in planes for H in [(L, i*L) for i in np.arange(-r, r, spacing)]]

hyperplanes = get_hyperplanes(
    *(rng.integers(-5,5,size=4) for _ in range(4)), spacing=0.02
#    *(rng.normal(size=4) for _ in range(4))
#    np.array([1,0,0,0]), np.array([0,1,0,0]), np.array([0,0,1,0]), np.array([0,0,0,1])
)

In [ ]:
P = Tope.from_vertices(v_36cell)
P.save_index()
#P.cut_2faces_with_hyperplanes(hyperplanes)
print("Decorations calculated. Unfolding...")
nets = plot_nets(P)

print("Got the nets!")

from tope.plot import save_figs_to_dir, save_figs_to_zip
save_figs_to_zip(nets, path="tmp/36-cell-test.zip", fmt="png", dpi=600)
#save_nets(P, "figs/4902603", force=True, fmt="svg")

In [ ]:
P = Tope.from_vertices(v_24cell)
P.save_index() # now must do this before passing to sub-topes
P.cut_2faces_with_hyperplanes(hyperplanes) # and this
save_nets(P, "figs/24cell", force=True)

In [ ]:
P = Tope.from_vertices(polys["2070"])
P.save_index() # now must do this before passing to sub-topes
save_nets(P, "figs/2070", force=True)

In [ ]:
#save_nets(Tope.from_vertices(polys["2070"]), "figs/2070", force=True)

In [ ]:
for name, poly in polys.items():
    pass # TODO. fix this
    #save_nets(Tope.from_vertices(poly), f"figs/{name}", force=True)

In [ ]:
with open("../polys1.json") as fd: data = json.load(fd)
v_bigboy = data["4902603"]